num1, op, num2 형식의 입력을 받아 학습을 통해 분류
현재 9, 9까지 가능

---



## Imports

In [ ]:
from platform import system

if system() == "Windows":
    %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
else:
    %pip install torch torchvision torchaudio

In [ ]:
%pip install matplotlib tqdm numpy pandas scipy jupyter ipywidgets
%pip install git+https://github.com/dAiv-CNU/torchdaiv.git

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
import torch.optim as optim
import random
from torch.utils.data import Dataset, DataLoader

# from torchdaiv.lectures.RNN import util

## Load Data

## Practice - MLP를 사용한 계산기 만들기

In [ ]:
# 사칙연산 계산 함수
def calculate(num1, op_idx, num2):
    if op_idx == 0:
        return num1 + num2
    elif op_idx == 1:
        return num1 - num2
    elif op_idx == 2:
        return num1 * num2
    elif op_idx == 3:
        return num1 // num2 if num2 != 0 else 0

In [ ]:
class ArithmeticDataset(Dataset):
    def __init__(self, num_samples=1000):
        super().__init__()
        self.data = []
        self.labels = []
        for _ in range(num_samples):
            num1 = random.randint(0, 9)
            num2 = random.randint(1, 9)  # 0 나눗셈 방지
            op_idx = random.randint(0, 3)
            x = torch.tensor([num1, op_idx, num2], dtype=torch.float32)
            y = calculate(num1, op_idx, num2)
            self.data.append(x)
            self.labels.append(torch.tensor([y], dtype=torch.float32))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [ ]:
# 모델 정의 (MLP)
class CalculateMLP(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=64, output_dim=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

calculatemlp = CalculateMLP()

In [ ]:
# 하이퍼파라미터 설정
MODEL_PARAMS = dict(
    input_dim=3,
    hidden_dim=64,
    output_dim=1  # 0 ~ 9
)

In [ ]:
# 데이터 로더
train_dataset = ArithmeticDataset(num_samples=2000)
test_dataset = ArithmeticDataset(num_samples=300)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CalculateMLP(**MODEL_PARAMS)
model.to(device)

CalculateMLP(
  (net): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [ ]:
epochs = 1000
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 학습 루프
for epoch in range(epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        logits = model(batch_x)
        loss = criterion(logits, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 100, Loss: 72.7563
Epoch 200, Loss: 15.2036
Epoch 300, Loss: 14.4463
Epoch 400, Loss: 10.0892
Epoch 500, Loss: 21.5921
Epoch 600, Loss: 0.5456
Epoch 700, Loss: 0.5388
Epoch 800, Loss: 1.5518
Epoch 900, Loss: 0.2187
Epoch 1000, Loss: 0.1305


In [ ]:
# 평가model.eval()
with torch.no_grad():
    test_x, test_y = next(iter(test_loader))
    test_x = test_x.to(device)
    test_y = test_y.to(device)

    preds = model(test_x).round()
    accuracy = (preds == test_y).float().mean()
    print(f"\nTest Accuracy: {accuracy.item() * 100:.2f}%\n")

    # 예시 출력
    op_symbol = ['+', '-', '*', '/']
    for i in range(10):
        input_expr = test_x[i].tolist()
        predicted = int(preds[i].item())
        target = int(test_y[i].item())
        print(f"Input: {int(input_expr[0])} {op_symbol[int(input_expr[1])]} {int(input_expr[2])} = Predicted: {predicted}, Target: {target}")



Test Accuracy: 67.33%

Input: 4 - 6 = Predicted: -2, Target: -2
Input: 3 / 6 = Predicted: 0, Target: 0
Input: 7 + 7 = Predicted: 14, Target: 14
Input: 6 * 1 = Predicted: 7, Target: 6
Input: 0 + 9 = Predicted: 9, Target: 9
Input: 8 - 9 = Predicted: -2, Target: -1
Input: 4 / 2 = Predicted: 3, Target: 2
Input: 5 / 8 = Predicted: 0, Target: 0
Input: 1 / 2 = Predicted: 0, Target: 0
Input: 9 + 4 = Predicted: 12, Target: 13


In [ ]:
op_map = {'+': 0, '-': 1, '*': 2, '/': 3}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("\n사칙연산 계산기 (종료하려면 'exit' 입력)")
model.eval()  # 루프 밖으로 이동
while True:
    user_input = input("입력 (예: 3 + 4): ")
    if user_input.lower() == 'exit':
        print("계산기를 종료합니다.")
        break

    try:
        # 공백 없는 입력도 처리 가능하도록 (예: "3+4")
        for op in op_map.keys():
            if op in user_input:
                parts = user_input.replace(op, f' {op} ').split()
                break
        else:
            raise ValueError("올바른 연산자가 없습니다.")

        num1, op, num2 = parts
        num1 = int(num1)
        num2 = int(num2)

        op_idx = op_map[op]
        x = torch.tensor([[num1, op_idx, num2]], dtype=torch.float32).to(device)

        with torch.no_grad():
            pred = model(x).item()
            pred_rounded = round(pred)

        print(f"예측 결과: {pred_rounded}\n")

    except Exception as e:
        print(f"입력 오류: {e}. 형식에 맞게 다시 입력하세요 (예: 3 + 4)\n")



사칙연산 계산기 (종료하려면 'exit' 입력)
입력 (예: 3 + 4): 5 * 5
예측 결과: 23



KeyboardInterrupt: Interrupted by user

위 계산기를 RNN으로 다시

In [ ]:
# 사칙연산 계산 함수
def calculate(num1, op, num2):
    if op == 0:
        return num1 + num2
    elif op == 1:
        return num1 - num2
    elif op == 2:
        return num1 * num2
    elif op == 3:
        return num1 // num2 if num2 != 0 else 0

In [ ]:
# 커스텀 Dataset 클래스
class ArithmeticDataset(Dataset):
    def __init__(self, num_samples=1000):
        self.data = []
        self.labels = []
        for _ in range(num_samples):
            num1 = random.randint(0, 9)
            num2 = random.randint(1, 9)  # 0 나눗셈 방지
            op = random.randint(0, 3)
            x = [[num1], [op], [num2]]
            y = calculate(num1, op, num2)
            y_shifted = y + 9  # -9 ~ 81 → 0 ~ 90
            self.data.append(torch.tensor(x, dtype=torch.float32))
            self.labels.append(torch.tensor(y_shifted, dtype=torch.long))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [ ]:
# 모델 정의(RNN)
class MathRNN(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, output_dim=91, num_layers=1):
        super().__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.rnn(x)
        out_last = out[:, -1, :]
        logits = self.fc(out_last)
        return logits

In [ ]:
# 하이퍼파라미터
input_dim = 1
hidden_dim = 64
output_dim = 91
epochs = 500
batch_size = 64

In [ ]:
# 데이터셋 및 데이터로더
train_dataset = ArithmeticDataset(num_samples=2000)
test_dataset = ArithmeticDataset(num_samples=300)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [ ]:
# 모델, 손실함수, 옵티마이저 정의
model = MathRNN(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 학습 루프
for epoch in range(epochs):
    for batch_x, batch_y in train_loader:
        logits = model(batch_x)
        loss = criterion(logits, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 10, Loss: 2.6448
Epoch 20, Loss: 2.4971
Epoch 30, Loss: 1.7586
Epoch 40, Loss: 1.1445
Epoch 50, Loss: 1.8271
Epoch 60, Loss: 1.4649
Epoch 70, Loss: 1.3601
Epoch 80, Loss: 1.0544
Epoch 90, Loss: 1.1311
Epoch 100, Loss: 0.8696
Epoch 110, Loss: 0.9073
Epoch 120, Loss: 0.5991
Epoch 130, Loss: 0.8833
Epoch 140, Loss: 0.7177
Epoch 150, Loss: 0.4073
Epoch 160, Loss: 0.5185
Epoch 170, Loss: 0.5421
Epoch 180, Loss: 0.5439
Epoch 190, Loss: 0.4995
Epoch 200, Loss: 0.5025
Epoch 210, Loss: 0.5955
Epoch 220, Loss: 0.3838
Epoch 230, Loss: 0.2751
Epoch 240, Loss: 0.3822
Epoch 250, Loss: 0.3236
Epoch 260, Loss: 0.3251
Epoch 270, Loss: 0.1859
Epoch 280, Loss: 0.3564
Epoch 290, Loss: 0.1489
Epoch 300, Loss: 0.1600
Epoch 310, Loss: 0.1436
Epoch 320, Loss: 0.1128
Epoch 330, Loss: 0.1515
Epoch 340, Loss: 0.0780
Epoch 350, Loss: 0.1011
Epoch 360, Loss: 0.0973
Epoch 370, Loss: 0.0754
Epoch 380, Loss: 0.1641
Epoch 390, Loss: 0.1040
Epoch 400, Loss: 0.0442
Epoch 410, Loss: 0.0538
Epoch 420, Loss: 0.0515
E

In [ ]:
# 평가
with torch.no_grad():
    for test_x, test_y in test_loader:
        preds = model(test_x).argmax(dim=1)
        preds_real = preds - 9
        test_real = test_y - 9
        accuracy = (preds_real == test_real).float().mean()
        print(f"\nTest Accuracy: {accuracy.item() * 100:.2f}%")

        # 예시 출력
        for i in range(10):
            input_expr = test_x[i].squeeze(-1).tolist()
            predicted = preds_real[i].item()
            target = test_real[i].item()
            op_symbol = ['+', '-', '*', '/'][int(input_expr[1])]
            print(f"Input: {int(input_expr[0])} {op_symbol} {int(input_expr[2])} = Predicted: {predicted}, Target: {target}")



Test Accuracy: 99.67%
Input: 4 * 3 = Predicted: 12, Target: 12
Input: 2 / 6 = Predicted: 0, Target: 0
Input: 0 * 2 = Predicted: 0, Target: 0
Input: 0 * 4 = Predicted: 0, Target: 0
Input: 3 - 9 = Predicted: -6, Target: -6
Input: 6 / 9 = Predicted: 0, Target: 0
Input: 8 - 5 = Predicted: 3, Target: 3
Input: 7 * 8 = Predicted: 56, Target: 56
Input: 3 / 7 = Predicted: 0, Target: 0
Input: 1 * 1 = Predicted: 1, Target: 1


In [ ]:
op_map = {'+': 0, '-': 1, '*': 2, '/': 3}

print("\n사칙연산 계산기 (종료하려면 'exit' 입력)")
while True:
    user_input = input("입력 (예: 3 + 4): ")
    if user_input.lower() == 'exit':
        print("계산기를 종료합니다.")
        break
    else:
        num1, op, num2 = user_input.strip().split()
        num1 = int(num1)
        num2 = int(num2)
        op_idx = op_map[op]
        x = torch.tensor([[[num1], [op_idx], [num2]]], dtype=torch.float32)
        model.eval()
        with torch.no_grad():
            pred = model(x).argmax(dim=1).item() - 9
        print(f"예측 결과: {pred}\n")


사칙연산 계산기 (종료하려면 'exit' 입력)
입력 (예: 3 + 4): 3 + 2
예측 결과: 5

입력 (예: 3 + 4): exit
계산기를 종료합니다.
